## Superconducting Transmon Device Design

Author: Roman Aseguinolaza Gallo

This notebook contains the design and fabrication details of a superconducting device featuring 8 transmons with varying capacitor geometries. The purpose of this study is to analyze the impact of interface defects on transmon coherence. The design follows these constraints:

- Each transmon is capacitively coupled to its own $\lambda/4$ resonator.
- The resonators are inductively coupled to a common feedline.
- Coplanar waveguide (CPW) geometry ensures a 50Ω impedance.
- The resonator frequencies range from 7 GHz to 8 GHz, spaced by at least 50 MHz.
- The transmon capacitors systematically vary in shape to study defect contributions.
- The design follows fabrication constraints, including minimum feature sizes and material properties.

The final design is provided in `.gds` format, using metal as the drawn layer. The notebook also includes calculations for resonator lengths, capacitances, and Josephson junction parameters.

<img src='pics/Broad.png' alt='Map' width='800'>

---

In [1]:
import gdspy as gds
import numpy as np


The substrate of the chip is 500 um-Silicon, which has a low-T dielectric constant, $\varepsilon_r=11.45$. The superconducting metal is aluminum, with thickness $t=100$ nm. Fabrication constraints say that there cannot be a feature smaller than 1 um, except for the Josephson junction, which has smaller features of up to 50 nm.

Each resonator must have a different frequency between 7-8 GHz, with at least 50 MHz of spacing between them. We choose frequencies varying from 7.08 Ghz to 7.92 Ghz, all 8 separated by 120 MHz. From these values, we obtain the length the $\lambda/4$ resonators must have, using:

\begin{equation*}
    l = \frac{\lambda}{4} = \frac{1}{4}\frac{c}{f_0\sqrt{\varepsilon_{eff}}},
\end{equation*}

where $l$ is the resonator length, $c$ is the speed of light, $f_0$ is the resonant frequency and $\varepsilon_{eff}=\frac{1+\varepsilon_r}{2}$ is the effective relative permittivity.

In [2]:
# Calculation of the lengths of the resonator for the given frequencies

f_0 = np.array([7.08 + i * 0.12 for i in range(8)])
l = 3e8 / (4 * f_0 * 1e9 * np.sqrt(6.225)) * 1e3

print("Resonator frequencies (GHz):", f_0)
print("Corresponding resonator lengths (mm):", np.round(l, 3))


Resonator frequencies (GHz): [7.08 7.2  7.32 7.44 7.56 7.68 7.8  7.92]
Corresponding resonator lengths (mm): [4.246 4.175 4.107 4.04  3.976 3.914 3.854 3.795]


The transmon must have a floating structure, and have a Manhattan structure junction. We aim at $\omega_q = 4$ Ghz and $\alpha = -300$ MHz. From these values, we can get to the area of the junction and the corresponding width of the metal. It follows from the following set of equations:

\begin{aligned}
\left\{
\begin{aligned}
\hbar \omega_q &= \sqrt{8\,E_J\,E_C} - E_C,\\
\hbar \alpha &\approx -\,E_c,\\
E_J &= \frac{\Phi_0}{2\pi}\,I_c = \frac{\Phi_0}{2\pi}\,J_c\,S
\end{aligned}
\right.
\quad \Longrightarrow \quad
S = \frac{h\,(\omega_q - \alpha)^2}{8\,|\alpha|\,\Phi_0\,J_c}
= 5163 \;\;\mathrm{nm}^2,
\end{aligned}

where we estimate that $J_C = 3$ uA/um $^2$. The width of the junction metal is therefore $\sqrt{S}=72$ nm. Certain parts are made wider in order to avoid the creation of other non desired junctions.

In [3]:
# Junction area calculation

h = 6.63e-34
hbar = h / (2 * np.pi)
e = 1.6e-19
w_q = 4e9 * 2 * np.pi
alpha = -300e6 * 2 * np.pi
phi_0 = h / (2 * e)
J_c = 3e6

S_juntion = -h * (w_q - alpha) ** 2 / (8 * alpha * phi_0 * J_c)
# Convert S_juntion from m^2 to nm^2
S_juntion_nm2 = S_juntion * 1e18
print(f"Manhattan junction area: {S_juntion_nm2:.0f} nm^2")


Manhattan junction area: 5163 nm^2


We estimate $C_J = S_CS = 0.258$ fF, where $S_C=50$ fF/um $^2$. With this capacitance known, we obtain the larger capacitance of the transmon $C=\frac{e^2}{2E_C}-C_J\approx64$ fF. In order to study the different contribution of the interface defects, each transmon must have a different capacitor geometry. We achieve this by setting the width of the capacitor pads to 50 um for all transmons, and changing the gap where the junction is located, $d$, and the length of the pads, $l$, accordingly using the following formula:

\begin{equation*}
    C = 500\;\frac{\varepsilon_0\varepsilon_{eff}\;l\;t}{d},
\end{equation*}

where $\varepsilon_0$ is the permittivity of vacuum. This is the common parallel plate capacitor formula, adjusted with a suitable prefactor in order to obtain reasonable estimates for the parameters. The methodology is a little bit hand wavy, but these must be simulated before fabrication anyways.

We set the distance from the transmon to the resonator using the same formula, aiming for a capacitance between them of 5 fF.

In [4]:
# Capacitor length proportion calculation

eps_0 = 8.85e-12
eps_r = 6.225
t = 100e-9

# Capacitance with the resonator
Cr_resonator = 5e-15
l_d_resonator = Cr_resonator / (eps_0 * eps_r * t) / 500
print(f"l/d proportion for the capacitance with the resonator: {l_d_resonator:.6f}")

# Capacitance for the transmon
Cr_transmon = 64e-15
l_d_transmon = Cr_transmon / (eps_0 * eps_r * t) / 500
print(f"l/d proportion for the capacitance for the transmon: {l_d_transmon:.6f}")


l/d proportion for the capacitance with the resonator: 1.815170
l/d proportion for the capacitance for the transmon: 23.234180


We accommodate all 8 transmons in a feedline with 4 mm of length, and coplanar waveguide geometry matched to 50 $\Omega$.  Its width must fall between 4-10 um. Using a CPW impedance calculator (https://www.microwaves101.com/calculators/864-coplanar-waveguide-calculator), we set a feedline width of 7 um and a gap between the feedline and ground of 4 um. This gives an impedance of 50.224 $\Omega$ using the mentioned calculator (not exactly 50 $\Omega$, but keeps the numbers clean).

The launcher width must be within 200-300 um. We set a width of 235 um and a gap of 125 um to have a 50.009 $\Omega$ impedance. We also set a transition with 100 um in length between feedline and launcher.

The width and gap of the coplanar waveguides on the resonator will be the same as those of the feedline.
The inductive coupling between the two must be of the same length for all resonators, within 250-400 um. We choose 300 um. The distance between the centers of both feedlines and resonators must be within 10-40 um, and we choose 20 um for this spacing.

In order to achieve the different resonator length, all meandering metal does the same number of turns, with the same turn radius of 64 um. Changing the length of the segment between the turns for each transmon sets them apart.

We end the resonator design by setting a capacitor length at the end of 100 um.

In [ ]:
# This dictionary will contain all relevant chip info
chip_info = {}

# chip params
chip_info['chip_size_x'], chip_info['chip_size_y'] = 5000, 5000

# feedline params
chip_info['feedline_width'] = 7
chip_info['feedline_gap'] = 4
chip_info['feedline_length'] = 4000

# launcher parameters
chip_info['launcher_width'] = 235
chip_info['launcher_gap'] = 125
chip_info['launcher_length'] = 235
chip_info['transition_length'] = 100

# resonator parameters
chip_info['resonator_width'] = 7
chip_info['resonator_gap'] = 4
chip_info['resonator_turn_radius'] = 64
chip_info['resonator_lengths'] = [4246, 4175, 4107, 4040, 3976, 3914, 3854, 3795]
chip_info['resonator_coupling lentgh'] = 300
chip_info['resonator_feedline_spacing'] = 20
chip_info['resonator_capacitor_length'] = 100
chip_info['resonator_positions'] = [-1900, -1400, -900, -400,  100,  600,  1100,  1600]

# qubit parameters
chip_info['qubit_gaps'] = [5, 6, 7, 8, 9, 10, 11, 12]
chip_info['qubit_capacitor_width'] = 50
chip_info['qubit_manhattan_width'] = 0.072

chip_info['resonator_l_over_d'] = 1.815
chip_info['qubit_l_over_d'] = 23.234


---

We generate the dielectric parts of the shape, which will be removed from the chip. The result will be the metallic part.

We now generate functions to draw the launcher, feedline and resonator dielectrics and qubit dielectrics, as well as the junction metal (which isn't drawn in the negative).

In [6]:
def get_chip_background(chip_info, layer):
    chip_size = [chip_info['chip_size_x'], chip_info['chip_size_y']]
    chip_background = gds.Rectangle((-chip_size[0]/2,-chip_size[1]/2), (chip_size[0]/2, chip_size[1]/2), layer=layer)
    return chip_background


<img src='pics/Feedline.png' alt='Map' width='600'>

In [7]:
def get_launcher_dielectric(chip_info, layer):

    # launcher parameters
    launcher_gap = chip_info['launcher_gap']
    launcher_width = chip_info['launcher_width']
    launcher_length = chip_info['launcher_length']
    transition_length = chip_info['transition_length']

    feedline_width = chip_info['feedline_width']
    feedline_gap = chip_info['feedline_gap']
    feedline_length = chip_info['feedline_length']

    # Let's generate the outer polygon
    outer_polygon = gds.Polygon([(0, -feedline_width/2 - feedline_gap),
                    (-transition_length, -launcher_width/2-launcher_gap),
                    (-transition_length-launcher_length-launcher_gap, -launcher_width/2-launcher_gap),
                    (-transition_length-launcher_length-launcher_gap, launcher_width/2+launcher_gap),
                    (-transition_length, launcher_width/2+launcher_gap),
                    (0, feedline_width/2+feedline_gap)])

    # Now the inner polygon
    inner_polygon = gds.Polygon([(0, feedline_width/2),
                    (-transition_length, launcher_width/2),
                    (-transition_length-launcher_length, launcher_width/2),
                    (-transition_length-launcher_length, -launcher_width/2),
                    (-transition_length, -launcher_width/2),
                    (0, -feedline_width/2)])

    # By substracting the inner from the outer polygon we'll get the dielectric. Then we can translate it to the correct position
    launcher_dielectric = gds.boolean(outer_polygon, inner_polygon, 'not', layer=layer).translate(-feedline_length/2,0)
    return launcher_dielectric


In [8]:
def get_feedline_dielectric(chip_info, layer):

    feedline_width = chip_info['feedline_width']
    feedline_gap = chip_info['feedline_gap']
    feedline_length = chip_info['feedline_length']

    feedline_dielectric = gds.FlexPath(
            [(-feedline_length/2,0),(feedline_length/2,0)],
            [feedline_gap, feedline_gap], feedline_gap + feedline_width , layer=layer
        )

    return feedline_dielectric


<img src='pics/Resonator.png' alt='Map' width='500'>

In [9]:
def get_resonator_dielectric(chip_info, resonator_number, layer, n_turns=10):

    resonator_width = chip_info['resonator_width']
    resonator_gap = chip_info['resonator_gap']
    resonator_turn_radius = chip_info['resonator_turn_radius']
    resonator_length = chip_info['resonator_lengths'][resonator_number]
    resonator_coupling_length = chip_info['resonator_coupling lentgh']
    resonator_feedline_spacing = chip_info['resonator_feedline_spacing']
    resonator_capacitor_length = chip_info['resonator_capacitor_length']

    segment_length = (resonator_length - np.pi*resonator_turn_radius*(n_turns+1/2) - resonator_coupling_length) / (n_turns-1/2)

    # Let's generate the meandering path of the resonator
    path_points = [(0,0), (resonator_coupling_length, 0)]

    turn_count = 0
    while turn_count < n_turns:
        if turn_count == n_turns-1:
            if turn_count % 2 == 0:
                path_points.append((path_points[-1][0] + resonator_turn_radius, path_points[-1][1]))
                path_points.append((path_points[-1][0], path_points[-1][1] + 2*resonator_turn_radius))
                path_points.append((path_points[-1][0] - 2*resonator_turn_radius - segment_length/2, path_points[-1][1]))
                path_points.append((path_points[-1][0], path_points[-1][1] + resonator_turn_radius))
                turn_count += 1
            else:
                path_points.append((path_points[-1][0] - resonator_turn_radius, path_points[-1][1]))
                path_points.append((path_points[-1][0], path_points[-1][1] + 2*resonator_turn_radius))
                path_points.append((path_points[-1][0] + 2*resonator_turn_radius + segment_length/2, path_points[-1][1]))
                path_points.append((path_points[-1][0], path_points[-1][1] + resonator_turn_radius))
                turn_count += 1
        elif turn_count % 2 == 0:
            path_points.append((path_points[-1][0] + resonator_turn_radius, path_points[-1][1]))
            path_points.append((path_points[-1][0], path_points[-1][1] + 2*resonator_turn_radius))
            path_points.append((path_points[-1][0] - resonator_turn_radius - segment_length, path_points[-1][1]))
            turn_count += 1
        else:
            path_points.append((path_points[-1][0] - resonator_turn_radius, path_points[-1][1]))
            path_points.append((path_points[-1][0], path_points[-1][1] + 2*resonator_turn_radius))
            path_points.append((path_points[-1][0] + resonator_turn_radius + segment_length, path_points[-1][1]))
            turn_count += 1

    meander_path_out = gds.FlexPath(path_points, resonator_width + 2*resonator_gap, corners="circular bend", bend_radius=resonator_turn_radius, gdsii_path=True, layer=layer)
    meander_path_in = gds.FlexPath(path_points, resonator_width, corners="circular bend", bend_radius=resonator_turn_radius, gdsii_path=True, layer=layer)

    # Now the capacitor at the end of the resonator
    capacitor_rect_out = gds.Rectangle((path_points[-1][0] - resonator_capacitor_length/2-resonator_gap, path_points[-1][1]), (path_points[-1][0] + resonator_capacitor_length/2+resonator_gap, path_points[-1][1] + resonator_width + 2*resonator_gap), layer=layer)
    capacitor_rect_in = gds.Rectangle((path_points[-1][0] - resonator_capacitor_length/2, path_points[-1][1] + resonator_gap), (path_points[-1][0] + resonator_capacitor_length/2, path_points[-1][1] + resonator_width + resonator_gap), layer=layer)
    capacitor_correction = gds.Rectangle((path_points[-1][0] - resonator_width/2, path_points[-1][1]), (path_points[-1][0] + resonator_width/2, path_points[-1][1] + resonator_gap), layer=layer)

    resonator_dielectric_0 = gds.boolean(meander_path_out, meander_path_in, 'not', layer=layer)
    resonator_dielectric_1 = gds.boolean(capacitor_rect_out, capacitor_rect_in, 'not', layer=layer)
    resonator_dielectric_2 = gds.boolean(resonator_dielectric_1, capacitor_correction, 'not', layer=layer)

    resonator_dielectric = gds.boolean(resonator_dielectric_0, resonator_dielectric_2, 'or', layer=layer)
    return resonator_dielectric, (path_points[-1]+(0,resonator_width+2*resonator_gap))


In [10]:
def get_label(text, layer, spacing_N=20, spacing_E=20, spacing_W=20):
    label = gds.Text(text, 3*spacing_N, (0, 0), layer=layer)
    box = gds.Rectangle((-1*spacing_E,0),(3*spacing_W,4*spacing_N), layer=layer)
    label = gds.boolean(box, label, 'not', layer=layer)
    return label


<img src='pics/Junction.png' alt='Map' width='600'>

In [11]:
def get_qubit_dielectric(chip_info, qubit_number, layer):

    qubit_gap = chip_info['qubit_gaps'][qubit_number]
    qubit_capacitor_width = chip_info['qubit_capacitor_width']
    resonator_capacitor_length = chip_info['resonator_capacitor_length']

    resonator_l_over_d = chip_info['resonator_l_over_d']
    qubit_l_over_d = chip_info['qubit_l_over_d']

    d_resonator = resonator_capacitor_length / resonator_l_over_d
    l_qubit = qubit_gap * qubit_l_over_d

    qubit_dielectric_bg = gds.Rectangle((-l_qubit/2-d_resonator, 0), (l_qubit/2+d_resonator, 2*d_resonator+2*qubit_capacitor_width+qubit_gap), layer=layer)

    capacitor_transmon = gds.Rectangle((-l_qubit/2,0),(l_qubit/2,qubit_capacitor_width), layer=layer).translate(0, d_resonator)
    capacitor_resonator = gds.Rectangle((-l_qubit/2,0),(l_qubit/2,qubit_capacitor_width), layer=layer).translate(0, d_resonator+qubit_capacitor_width+qubit_gap)
    
    qubit_dielectric = gds.boolean(qubit_dielectric_bg, capacitor_resonator, 'not', layer=layer)
    qubit_dielectric = gds.boolean(qubit_dielectric, capacitor_transmon, 'not', layer=layer)

    return qubit_dielectric


In [12]:
def get_junction_metal(chip_info, qubit_number, layer):

    qubit_gap = chip_info['qubit_gaps'][qubit_number]
    qubit_capacitor_width = chip_info['qubit_capacitor_width']
    qubit_manhattan_width = chip_info['qubit_manhattan_width']
    resonator_capacitor_length = chip_info['resonator_capacitor_length']

    resonator_l_over_d = chip_info['resonator_l_over_d']
    qubit_l_over_d = chip_info['qubit_l_over_d']

    d_resonator = resonator_capacitor_length / resonator_l_over_d
    l_qubit = qubit_gap * qubit_l_over_d

    bottom_line = gds.Rectangle((-qubit_manhattan_width*2,0),(qubit_manhattan_width*2,qubit_gap/2), layer=layer).translate(8*qubit_manhattan_width, d_resonator + qubit_capacitor_width)

    top_line = gds.Rectangle((-qubit_manhattan_width/2,0),(qubit_manhattan_width/2,qubit_gap/2+5*qubit_manhattan_width), layer=layer).translate(-5*qubit_manhattan_width, d_resonator + qubit_capacitor_width + qubit_gap/2-5*qubit_manhattan_width)

    middle_line = gds.Rectangle((-10*qubit_manhattan_width,-qubit_manhattan_width/2),(10*qubit_manhattan_width,qubit_manhattan_width/2), layer=layer).translate(0, d_resonator + qubit_capacitor_width + qubit_gap/2)
    
    bottom_pad = gds.Rectangle((-qubit_gap/8,-qubit_gap/4),(qubit_gap/8,qubit_gap/4), layer=layer).translate(8*qubit_manhattan_width, d_resonator + qubit_capacitor_width)

    top_pad = gds.Rectangle((-qubit_gap/8,-qubit_gap/4),(qubit_gap/8,qubit_gap/4), layer=layer).translate(-5*qubit_manhattan_width, d_resonator + qubit_capacitor_width + qubit_gap)

    junction_metal = gds.boolean(bottom_line, top_line, 'or', layer=layer)
    junction_metal = gds.boolean(junction_metal, middle_line, 'or', layer=layer)
    junction_metal = gds.boolean(junction_metal, bottom_pad, 'or', layer=layer)
    junction_metal = gds.boolean(junction_metal, top_pad, 'or', layer=layer)

    return junction_metal


---

Each component is created using the corresponding function and added to the main mask cell (`MaskCell`). The dielectric parts are subtracted from the background layer to create the final Metal design. The layout is then saved as a GDSII file and optionally visualized using the `gdspy.LayoutViewer`.

We draw all the area covered by dielectric and the metal on the final device on different masks. We also draw the Josephson junctions separately on their own positive mask. On the written `.gds` file, the metal is drawn in layer 0, the dielectric in layer 40, and the junctions in level 80.


In [13]:
# Generate a library so that you create all you elements from this library
lib = gds.GdsLibrary()

# Create the mask cell where everything will be contained
MaskCell = lib.new_cell('Mask')

# Also generate a list with all the dielectric (negatives) that will be substracted from the main chip
negatives_dielectric = []


# CHIP BACKGROUND
chip_background = get_chip_background(chip_info, 0)


# LAUNCHER DIELECTRIC in the dielectric/negative layer 1
launcher_dielectric_0 = lib.new_cell('Launcher_0')
launcher_dielectric_0.add(get_launcher_dielectric(chip_info, 40))
MaskCell.add(launcher_dielectric_0)
# Also let's add the reference to this cell to the list of negative cells to be removed from the chip background
launcher_dielectric_cell_ref_0 = gds.CellReference(launcher_dielectric_0)
negatives_dielectric.append(launcher_dielectric_cell_ref_0)

# Rotated launcher
launcher_dielectric_1 = lib.new_cell('Launcher_1')
launcher_dielectric_1.add(get_launcher_dielectric(chip_info, 40).rotate(np.pi))
MaskCell.add(launcher_dielectric_1)
# Also let's add the reference to this cell to the list of negative cells to be removed from the chip background
launcher_dielectric_cell_ref_1 = gds.CellReference(launcher_dielectric_1)
negatives_dielectric.append(launcher_dielectric_cell_ref_1)


# FEEDLINE DIELECTRIC
feedline_dielectric = lib.new_cell('Feedline')
feedline_dielectric.add(get_feedline_dielectric(chip_info, 40))
feedline_cell_ref = gds.CellReference(feedline_dielectric)
negatives_dielectric.append(feedline_cell_ref)
MaskCell.add(feedline_dielectric)


# RESONATOR DIELECTRIC
qubit_positions = [] # This list will store the positions of the qubits at the end of the resonators
for i in range(len(chip_info['resonator_lengths'])):
    resonator_dielectric = lib.new_cell('Resonator_'+str(i))
    resonator_positions = chip_info['resonator_positions']
    resonator_feedline_spacing = chip_info['resonator_feedline_spacing']
    if i % 2 == 0:
        resonator, point = get_resonator_dielectric(chip_info, i, 40)
        resonator_dielectric.add(resonator.translate(resonator_positions[i], resonator_feedline_spacing))
        resonator_dielectric.add(get_label(str(i), 40).translate(resonator_positions[i], 3*resonator_feedline_spacing))
        qubit_positions.append(point)
    else:
        resonator, point = get_resonator_dielectric(chip_info, i, 40)
        resonator_dielectric.add(resonator.mirror((0,0),(1,0)).translate(resonator_positions[i], -resonator_feedline_spacing))
        resonator_dielectric.add(get_label(str(i), 40).translate(resonator_positions[i], -6*resonator_feedline_spacing))
        qubit_positions.append(point)
    resonator_cell_ref = gds.CellReference(resonator_dielectric)
    negatives_dielectric.append(resonator_cell_ref)
    MaskCell.add(resonator_dielectric)


# QUBIT DIELECTRIC
for i in range(len(qubit_positions)):
    qubit_dielectric = lib.new_cell('Qubit_'+str(i))
    qubit_gaps = chip_info['qubit_gaps']
    resonator_positions = chip_info['resonator_positions']
    resonator_feedline_spacing = chip_info['resonator_feedline_spacing']
    resonator_width = chip_info['resonator_width']
    resonator_gap = chip_info['resonator_gap']
    if i % 2 == 0:
        qubit = get_qubit_dielectric(chip_info, i, 40)
        qubit_dielectric.add(qubit.translate(resonator_positions[i], resonator_feedline_spacing+resonator_width+resonator_gap).translate(qubit_positions[i][0], qubit_positions[i][1]))
    else:
        qubit = get_qubit_dielectric(chip_info, i, 40)
        qubit_dielectric.add(qubit.translate(resonator_positions[i], resonator_feedline_spacing+resonator_width+resonator_gap).translate(qubit_positions[i][0], qubit_positions[i][1]).mirror((0,0),(1,0)))
    qubit_cell_ref = gds.CellReference(qubit_dielectric)
    negatives_dielectric.append(qubit_cell_ref)
    MaskCell.add(qubit_dielectric)


# JUNCTION LAYER
for i in range(len(qubit_positions)):
    junction_metal = lib.new_cell('Junction '+str(i))
    qubit_gaps = chip_info['qubit_gaps']
    resonator_positions = chip_info['resonator_positions']
    resonator_feedline_spacing = chip_info['resonator_feedline_spacing']
    resonator_width = chip_info['resonator_width']
    resonator_gap = chip_info['resonator_gap']
    if i % 2 == 0:
        junction = get_junction_metal(chip_info, i, 80)
        junction_metal.add(junction.translate(resonator_positions[i], resonator_feedline_spacing+resonator_width+resonator_gap).translate(qubit_positions[i][0], qubit_positions[i][1]))
    else:
        junction = get_junction_metal(chip_info, i, 80)
        junction_metal.add(junction.translate(resonator_positions[i], resonator_feedline_spacing+resonator_width+resonator_gap).translate(qubit_positions[i][0], qubit_positions[i][1]).mirror((0,0),(1,0)))
    MaskCell.add(junction_metal)

# Chip identification
chip_id = lib.new_cell('Chip ID')
chip_id.add(get_label('INTERFACE DEFECT TEST', 40, 20, 20, 380).translate(-2000,2000))
chip_id_cell_ref = gds.CellReference(chip_id)
negatives_dielectric.append(chip_id_cell_ref)
MaskCell.add(chip_id)


# METALLIC LAYER (removing the dielectric from the full chip bakcground)
Metal = lib.new_cell('Metal')

metal_layer = chip_background
for negative in negatives_dielectric:
    metal_layer = gds.boolean(metal_layer, [negative], 'not', max_points=1000000, layer=0)
Metal.add(metal_layer)
MaskCell.add(gds.CellReference(Metal, origin=(0, 0)))

lib.write_gds('SC_Device.gds')


C:\Users\roman\AppData\Local\Temp\ipykernel_10968\49969062.py:113: UserWarning: [GDSPY] Polygons with more than 8190 are not supported by the official GDSII specification.  This GDSII file might not be compatible with all readers.
  lib.write_gds('SC_Device.gds')


In [ ]:
''' To visualize the layout, uncomment the following line '''
# gds.LayoutViewer(library=lib)

''' Optionally, save an image of the cell as SVG. '''
# Metal.write_svg('SC_Device.svg')


<img src='pics/SC_Device.svg' alt='Map' width='800'>